[Source](https://pytorch.org/blog/Accelerating-Hugging-Face-and-TIMM-models/)

In [4]:
%%capture
!pip3 install --pre torch --extra-index-url https://download.pytorch.org/whl/nightly/cpu
!pip3 install transformers

In [16]:
import torch
from transformers import BertTokenizer, BertModel
import time


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
slow_model = BertModel.from_pretrained("bert-base-uncased")
fast_model = torch.compile(slow_model)
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors="pt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
for _ in range(10):
    st_time = time.time()
    output = slow_model(**encoded_input)
    print(time.time() - st_time)

0.16397953033447266
0.1758866310119629
0.1841731071472168
0.16378402709960938
0.1588287353515625
0.16391873359680176
0.14704632759094238
0.14936327934265137
0.16373395919799805
0.14490866661071777


 The first run is slow and that’s because the model is being compiled. Subsequent runs will be faster so it’s common practice to warm up your model before you start benchmarking it.

In [18]:
for _ in range(10):
    st_time = time.time()
    output = fast_model(**encoded_input)
    print(time.time() - st_time)

32.198747873306274
0.0909581184387207
0.0881953239440918
0.096771240234375
0.08929443359375
0.09436845779418945
0.09152889251708984
0.09056305885314941
0.09667491912841797
0.09199810028076172
